In [1]:
import numpy as np
import pandas as pd

In [2]:
addrs = [
    "Ьмбшмщъоэцфх ысь.р.37 цо.399",
    "юя. № 4914у.14 щс.284",
    "2-ф Дгяегчжяхф иа.щ.73 яч.70",
    "87-р су УСЗЛл.16 сй.158",
    "1-ъ Васвяюсьщуыяувыщъ ацб.х.35 ыу.167",
    "шр. Межуручбкй.32 пз.476",
    "Адэуьфьащчш бвю.у.37 щс.368",
    "Ьужучузцпед шр.й.57 пз.41",
    "Баэчфтс еэ.ц.68 ьф.257",
    "Иугфцки ыу.м.76 тк.49",
]

emails = [
    "vaeqbt52@symux.oay",
    "zxtwc.hxcrtgt@vbpxa.rdb",
    "zgzxov.cdgg@ojmkct.xjh",
    "yohflz@nsvcly.jvt",
    "wfiu.cvsjrtb@yreu.tfd",
    "rnpfdqf56@mtyrfnq.htr",
    "ypobnct.qdnat@vddslxc.dgv",
    "rpqnsl@bnxtep.htr",
    "pujggck@zglesad.uge",
    "mzqjmzbw.tqvl@ouiqt.kwu",
]

df = pd.DataFrame({
    "email": emails,
    "address": addrs,
})
df.head()

,email,address
0,vaeqbt52@symux.oay,Ьмбшмщъоэцфх ысь.р.37 цо.399
1,zxtwc.hxcrtgt@vbpxa.rdb,юя. № 4914у.14 щс.284
2,zgzxov.cdgg@ojmkct.xjh,2-ф Дгяегчжяхф иа.щ.73 яч.70
3,yohflz@nsvcly.jvt,87-р су УСЗЛл.16 сй.158
4,wfiu.cvsjrtb@yreu.tfd,1-ъ Васвяюсьщуыяувыщъ ацб.х.35 ыу.167


In [3]:
alphabet_ru = set("абвгдежзийклмнопрстуфхцчшщъыьэюя")
alphabet_en = set("abcdefghijklmnopqrstuvwxyz")
len(alphabet_ru), len(alphabet_en)

(32, 26)

Один из популярных простых алгоритмов шифрования - это шифр Цезаря: каждая буква сдвигается на одно и то же число:  
$$
num_{new} = (num_{old} + k) \,\, mod \,\, |alphabet|
$$,  
где $num_{new}$ и $num_{old}$ - соответственно, номера исходной буквы и зашифрованной.  
Нетрудно заметить, что в адресах перед последним числом везде стоят 2 буквы с разницей номеров 8 - как у "*кв*" (квартира) - это поможет найти сдвиг.  
В случае с почтой ключ шифрования можно найти с помошью последних трёх букв: они являются сдвигами популярного домена "*com*".

In [20]:
def get_shift_addr(addr):
    last_word = list(addr.lower().strip("0123456789. \n").split())[-1]
    return ord(last_word[0]) - ord('к')


def get_shift_mail(email):
    last_word = list(email.lower().strip("").split("."))[-1]
    if (ord(last_word[0]) - ord('c') + 26) % 26 \
        == (ord(last_word[1]) - ord('o') + 26) % 26 \
            == (ord(last_word[2]) - ord('m') + 26) % 26:
        return ord(last_word[0]) - ord('c')  # this case, email is "...@<...>.com"
    else:
        return ord(last_word[0]) - ord('o')  # this case, email is "...@<...>.org"

In [21]:
def shift_letter(let, shift, dom):
    if dom not in {"mail", "addr"}:
        return let
    alphabet = alphabet_ru if dom == "addr" else alphabet_en
    if let.lower() not in alphabet:
        return let
    first_let = 'а' if dom == "addr" else 'a'
    new_let = chr((ord(let.lower()) - ord(first_let) + len(alphabet) - shift) % len(alphabet) + ord(first_let))
    return new_let if not let.isupper() else new_let.upper()


def decipher(original, dom):
    if dom not in {"mail", "addr"}:
        return original
    shift = get_shift_addr(original) if dom == "addr" else get_shift_mail(original)
    return "".join([shift_letter(let, shift, dom) for let in original])


def get_cipher_key(original, dom):
    if dom not in {"mail", "addr"}:
        return ""
    shift = get_shift_addr(original) if dom == "addr" else get_shift_mail(original)
    first_let = 'а' if dom == "addr" else 'a'
    last_let = 'я' if dom == "addr" else 'z'
    return "".join([shift_letter(chr(let), -shift, dom) for let in range(ord(first_let), ord(last_let) + 1)])

In [22]:
df["email_key"] = df["email"].apply(lambda mail: get_cipher_key(mail, "mail"))
df["email_deciphered"] = df["email"].apply(lambda mail: decipher(mail, "mail"))
df["address_key"] = df["address"].apply(lambda addr: get_cipher_key(addr, "addr"))
df["address_deciphered"] = df["address"].apply(lambda addr: decipher(addr, "addr"))
df.head(10)

,email,address,email_key,email_deciphered,address_key,address_deciphered
0,vaeqbt52@symux.oay,Ьмбшмщъоэцфх ысь.р.37 цо.399,mnopqrstuvwxyzabcdefghijkl,joseph52@gmail.com,мнопрстуфхцчшщъыьэюяабвгдежзийкл,Рахмановский пер.д.37 кв.399
1,zxtwc.hxcrtgt@vbpxa.rdb,юя. № 4914у.14 щс.284,pqrstuvwxyzabcdefghijklmno,kiehn.sincere@gmail.com,прстуфхцчшщъыьэюяабвгдежзийклмно,пр. № 4914д.14 кв.284
2,zgzxov.cdgg@ojmkct.xjh,2-ф Дгяегчжяхф иа.щ.73 яч.70,vwxyzabcdefghijklmnopqrstu,electa.hill@torphy.com,хцчшщъыьэюяабвгдежзийклмнопрстуф,2-я Покровская ул.д.73 кв.70
3,yohflz@nsvcly.jvt,87-р су УСЗЛл.16 сй.158,hijklmnopqrstuvwxyzabcdefg,rhayes@glover.com,зийклмнопрстуфхцчшщъыьэюяабвгдеж,87-й км МКАДд.16 кв.158
4,wfiu.cvsjrtb@yreu.tfd,1-ъ Васвяюсьщуыяувыщъ ацб.х.35 ыу.167,rstuvwxyzabcdefghijklmnopq,ford.lebsack@hand.com,стуфхцчшщъыьэюяабвгдежзийклмнопр,1-й Спасоналивковский пер.д.35 кв.167
5,rnpfdqf56@mtyrfnq.htr,шр. Межуручбкй.32 пз.476,fghijklmnopqrstuvwxyzabcde,mikayla56@hotmail.com,ежзийклмнопрстуфхцчшщъыьэюяабвгд,ул. Заболотьед.32 кв.476
6,ypobnct.qdnat@vddslxc.dgv,Адэуьфьащчш бвю.у.37 щс.368,pqrstuvwxyzabcdefghijklmno,jazmyne.boyle@goodwin.org,прстуфхцчшщъыьэюяабвгдежзийклмно,Сходненский туп.д.37 кв.368
7,rpqnsl@bnxtep.htr,Ьужучузцпед шр.й.57 пз.41,fghijklmnopqrstuvwxyzabcde,mkling@wisozk.com,ежзийклмнопрстуфхцчшщъыьэюяабвгд,Чоботовская ул.д.57 кв.41
8,pujggck@zglesad.uge,Баэчфтс еэ.ц.68 ьф.257,stuvwxyzabcdefghijklmnopqr,xcrooks@hotmail.com,туфхцчшщъыьэюяабвгдежзийклмнопрс,Полевая ул.д.68 кв.257
9,mzqjmzbw.tqvl@ouiqt.kwu,Иугфцки ыу.м.76 тк.49,ijklmnopqrstuvwxyzabcdefgh,eriberto.lind@gmail.com,ийклмнопрстуфхцчшщъыьэюяабвгдежз,Алымова ул.д.76 кв.49


Ключ шифрования: это последовательность букв длиной с размер алфавита, где на месте каждой буквы стоит буква, на которую она заменяется при шифровании.

In [23]:
df.to_csv("task3_results.csv", index=False)